In [1]:
import os
import json
import requests
import matplotlib.pyplot as plt

In [2]:
OPENWEATHERMAP_API_KEY = "429e6570bf176b57734d33f03e17ea4a"

In [3]:
# --- Cell 2: Weather Fetcher Function ---

def fetch_weather_data(city: str, api_key: str) -> dict | None:
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
        "units": "metric"
    }

    try:
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        temperature = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        weather_description = data["weather"][0]["description"]
        wind_speed = data["wind"]["speed"]

        return {
            "city": city,
            "temperature": temperature,
            "humidity": humidity,
            "description": weather_description,
            "wind_speed": wind_speed
        }

    except requests.exceptions.HTTPError as e:
        if response.status_code == 401:
            print(f"Error: Invalid API key or unauthorized request for {city}. Details: {e}")
        elif response.status_code == 404:
            print(f"Error: City '{city}' not found. Details: {e}")
        else:
            print(f"HTTP error occurred for {city}: {e}")
    except requests.exceptions.ConnectionError as e:
        print(f"Network error occurred for {city}: {e}. Check your internet connection.")
    except requests.exceptions.Timeout:
        print(f"Request timed out for {city}. The server took too long to respond.")
    except requests.exceptions.RequestException as e:
        print(f"An unexpected request error occurred for {city}: {e}")
    except KeyError as e:
        print(f"Error parsing API response for {city}: Missing key {e}. Response structure might have changed.")
    except Exception as e:
        print(f"An unexpected error occurred while fetching data for {city}: {e}")

    return None


In [4]:
# --- Cell 3: Data Handler Functions ---

def read_cities_from_file(filepath: str) -> list[str]:
    cities = []
    if not os.path.exists(filepath):
        print(f"Error: Input cities file '{filepath}' not found.")
        return cities
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            for line in f:
                city = line.strip()
                if city:
                    cities.append(city)
    except IOError as e:
        print(f"Error reading cities file {filepath}: {e}")
    return cities

def save_weather_data(data: list[dict], filepath: str):
    try:
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4)
    except IOError as e:
        print(f"Error saving weather data to {filepath}: {e}")
    except TypeError as e:
        print(f"Error serializing data to JSON for {filepath}: {e}. Check data format.")

def load_weather_data(filepath: str) -> list[dict] | None:
    if not os.path.exists(filepath):
        print(f"Error: Data file '{filepath}' not found.")
        return None
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
            if isinstance(data, list):
                return data
            else:
                print(f"Error: Expected a list in JSON file, but got {type(data)}.")
                return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from {filepath}: {e}. File might be corrupted.")
    except IOError as e:
        print(f"Error loading weather data from {filepath}: {e}")
    return None

In [5]:
# --- Cell 4: Analyzer Function ---

def analyze_weather_data(weather_data: list[dict]) -> dict:
    if not weather_data:
        return {
            "total_cities": 0,
            "highest_temp_city": "N/A",
            "highest_temperature": "N/A",
            "lowest_temp_city": "N/A",
            "lowest_temperature": "N/A",
            "average_temperature": "N/A",
            "clear_sky_cities": [],
            "rain_cities": []
        }

    total_cities = len(weather_data)
    temperatures = [d["temperature"] for d in weather_data]

    highest_temp = -float('inf')
    highest_temp_city = ""
    lowest_temp = float('inf')
    lowest_temp_city = ""

    for data in weather_data:
        temp = data["temperature"]
        city = data["city"]
        if temp > highest_temp:
            highest_temp = temp
            highest_temp_city = city
        if temp < lowest_temp:
            lowest_temp = temp
            lowest_temp_city = city

    average_temperature = sum(temperatures) / len(temperatures)

    clear_sky_cities = []
    rain_cities = []
    for data in weather_data:
        description = data["description"].lower()
        city = data["city"]
        if "clear" in description or "sky" in description:
            clear_sky_cities.append(city)
        if "rain" in description or "drizzle" in description or "shower" in description:
            rain_cities.append(city)

    return {
        "total_cities": total_cities,
        "highest_temp_city": highest_temp_city,
        "highest_temperature": highest_temp,
        "lowest_temp_city": lowest_temp_city,
        "lowest_temperature": lowest_temp,
        "average_temperature": average_temperature,
        "clear_sky_cities": clear_sky_cities,
        "rain_cities": rain_cities
    }

In [6]:
# --- Cell 5: Reporter Function ---

def generate_report(insights: dict, report_filepath: str):
    report_content = []
    report_content.append("Weather Summary Report")
    report_content.append("=" * 25)
    report_content.append(f"Cities Processed: {insights['total_cities']}")
    report_content.append(f"Highest Temperature: {insights['highest_temperature']:.1f}°C - {insights['highest_temp_city']}")
    report_content.append(f"Lowest Temperature: {insights['lowest_temperature']:.1f}°C - {insights['lowest_temp_city']}")
    report_content.append(f"Average Temperature: {insights['average_temperature']:.1f}°C")

    report_content.append("\nClear Weather Cities:")
    if insights['clear_sky_cities']:
        for city in insights['clear_sky_cities']:
            report_content.append(f"- {city}")
    else:
        report_content.append("  None")

    report_content.append("\nRain in Cities:")
    if insights['rain_cities']:
        for city in insights['rain_cities']:
            report_content.append(f"- {city}")
    else:
        report_content.append("  None")

    try:
        with open(report_filepath, "w", encoding="utf-8") as f:
            f.write("\n".join(report_content))
    except IOError as e:
        print(f"Error writing report to {report_filepath}: {e}")

In [7]:
# --- Cell 6: Plotter Function ---

def plot_temperatures(weather_data: list[dict], output_filename="temperature_chart.png"):
    if not weather_data:
        print("No data available to plot temperatures.")
        return

    cities = [d["city"] for d in weather_data]
    temperatures = [d["temperature"] for d in weather_data]

    plt.figure(figsize=(12, 6))
    plt.bar(cities, temperatures, color='skyblue')
    plt.xlabel("City")
    plt.ylabel("Temperature (°C)")
    plt.title("Current Temperatures in Cities")
    plt.xticks(rotation=45, ha="right")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    try:
        plt.savefig(output_filename)
        print(f"Temperature chart saved as '{output_filename}'")
    except Exception as e:
        print(f"Error saving plot to {output_filename}: {e}")
    finally:
        plt.close()

In [8]:
# --- Cell 7: Main Execution Logic ---

def run_weather_analysis(
    cities_file: str = "cities.txt",
    output_data_file: str = "weather_data.json",
    report_file: str = "summary_report.txt",
    plot_flag: bool = False
):
    if OPENWEATHERMAP_API_KEY == "YOUR_OPENWEATHERMAP_API_KEY":
        print("WARNING: Please replace 'YOUR_OPENWEATHERMAP_API_KEY' in Cell 1 with your actual API key.")
        print("The program might not work correctly without a valid API key.")

    print(f"Reading cities from: {cities_file}")
    cities = read_cities_from_file(cities_file)
    if not cities:
        print(f"No cities found in {cities_file}. Exiting.")
        return

    print("Fetching weather data...")
    all_weather_data = []
    for city in cities:
        print(f"  Fetching data for {city}...")
        data = fetch_weather_data(city, OPENWEATHERMAP_API_KEY)
        if data:
            all_weather_data.append(data)
        else:
            print(f"  Could not fetch data for {city}. Skipping.")

    if not all_weather_data:
        print("No weather data was successfully fetched. Exiting.")
        return

    print(f"Saving weather data to: {output_data_file}")
    save_weather_data(all_weather_data, output_data_file)

    print("Analyzing weather data...")
    insights = analyze_weather_data(all_weather_data)

    print(f"Generating summary report to: {report_file}")
    generate_report(insights, report_file)

    if plot_flag:
        print("Generating temperature plot...")
        plot_temperatures(all_weather_data)
        print("Temperature chart saved as temperature_chart.png")

    print("\nWeather analysis complete!")
    print(f"Report saved to: {report_file}")
    print(f"Raw data saved to: {output_data_file}")

In [9]:
# --- Cell 8: How to Run in Jupyter ---

with open("my_cities.txt", "w") as f:
    f.write("London\n")
    f.write("Paris\n")
    f.write("Tokyo\n")
    f.write("New York\n")
    f.write("Delhi\n")

run_weather_analysis(cities_file="my_cities.txt", plot_flag=True)

Reading cities from: my_cities.txt
Fetching weather data...
  Fetching data for London...
  Fetching data for Paris...
  Fetching data for Tokyo...
  Fetching data for New York...
  Fetching data for Delhi...
Saving weather data to: weather_data.json
Analyzing weather data...
Generating summary report to: summary_report.txt
Generating temperature plot...
Temperature chart saved as 'temperature_chart.png'
Temperature chart saved as temperature_chart.png

Weather analysis complete!
Report saved to: summary_report.txt
Raw data saved to: weather_data.json
